# **Task 1: Data Ingestion and  Data Preprocessing**

In [15]:
import os
import sys
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('..')))
from telethon import TelegramClient
from dotenv import load_dotenv

# Load the data

In [17]:
# Load environment variables once
data = pd.read_csv('../data/telegram_data.csv',low_memory=False, index_col=False)

In [21]:
data

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14844,ክቡራን ደምበኞቻችን ዛሬ ገርጂ እና ብስራተ ገብሬል የሚገኘው ሱቃችን ሙሉ...,2024-09-27 08:02:52+00:00,photos/@sinayelj_14844.jpg
1,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14843,ለመላው የክርስትና እምነት ተከታዮች በሙሉ እንኳን ለመስቀል ደመራ በዓል ...,2024-09-27 07:38:57+00:00,photos/@sinayelj_14843.jpg
2,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14841,NaN,2024-09-26 12:24:51+00:00,photos/@sinayelj_14841.jpg
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14840,NaN,2024-09-26 12:24:51+00:00,photos/@sinayelj_14840.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14839,ለኮንዶሚኒየም ለጠባብ ቤቶች ገላግሌ የሆነ ከንፁህ የሲልከን ጥሬ እቃ የተ...,2024-09-26 12:24:51+00:00,photos/@sinayelj_14839.jpg
...,...,...,...,...,...,...
1064,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,13587,Kids Scooter \n\nስኩተሩ ቢፈጥን ፍጥነቱን መቆጣጠሪያ ፍሬን ያለ...,2024-02-17 12:31:00+00:00,photos/@sinayelj_13587.jpg
1065,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,13586,NaN,2024-02-16 17:12:32+00:00,photos/@sinayelj_13586.jpg
1066,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,13585,NaN,2024-02-16 17:06:07+00:00,photos/@sinayelj_13585.jpg
1067,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,13584,NaN,2024-02-16 17:06:07+00:00,photos/@sinayelj_13584.jpg


In [22]:
data.dropna()

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14844,ክቡራን ደምበኞቻችን ዛሬ ገርጂ እና ብስራተ ገብሬል የሚገኘው ሱቃችን ሙሉ...,2024-09-27 08:02:52+00:00,photos/@sinayelj_14844.jpg
1,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14843,ለመላው የክርስትና እምነት ተከታዮች በሙሉ እንኳን ለመስቀል ደመራ በዓል ...,2024-09-27 07:38:57+00:00,photos/@sinayelj_14843.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14839,ለኮንዶሚኒየም ለጠባብ ቤቶች ገላግሌ የሆነ ከንፁህ የሲልከን ጥሬ እቃ የተ...,2024-09-26 12:24:51+00:00,photos/@sinayelj_14839.jpg
8,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14835,Baby romper ከላዩ ፈር ውስጡ ኮተን የሆነ\n\n2000 birr\...,2024-09-26 12:24:11+00:00,photos/@sinayelj_14835.jpg
11,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14832,Chicco 5 in 1 mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\...,2024-09-26 12:20:22+00:00,photos/@sinayelj_14832.jpg
...,...,...,...,...,...,...
1044,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,13617,ክቡራን ደምበኞቻችን እሁድ ገርጂ እና ብስራተ ገብሬል የሚገኘው ሱቃችን ከ...,2024-02-18 02:34:15+00:00,photos/@sinayelj_13617.jpg
1050,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,13611,Kids sofa \n\nመታጠፍ መዘርጋት የሚችል ፊልም ለማየት ለማንበብ ...,2024-02-17 13:42:30+00:00,photos/@sinayelj_13611.jpg
1055,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,13606,Kids sofa 4400 birr \n\nFree delivery \n\n090...,2024-02-17 13:23:30+00:00,photos/@sinayelj_13606.jpg
1059,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,13602,Kids sofa \n 4000 birr \n\nFree delivery \n\n0...,2024-02-17 13:19:22+00:00,photos/@sinayelj_13602.jpg


In [23]:
print("Checking for NaN values in the 'Message' column:")
nan_count = data['Message'].isnull().sum()
print(f"Number of NaN values in 'Message' column: {nan_count}")

Checking for NaN values in the 'Message' column:
Number of NaN values in 'Message' column: 781


In [24]:
data = data.dropna(subset=['Message'])

# Print the shape of the dataset after dropping NaN values in the "Message" column
print(f"Dataset shape after dropping NaN values in 'Message' column: {data.shape}")

Dataset shape after dropping NaN values in 'Message' column: (288, 6)


In [25]:
data.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14844,ክቡራን ደምበኞቻችን ዛሬ ገርጂ እና ብስራተ ገብሬል የሚገኘው ሱቃችን ሙሉ...,2024-09-27 08:02:52+00:00,photos/@sinayelj_14844.jpg
1,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14843,ለመላው የክርስትና እምነት ተከታዮች በሙሉ እንኳን ለመስቀል ደመራ በዓል ...,2024-09-27 07:38:57+00:00,photos/@sinayelj_14843.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14839,ለኮንዶሚኒየም ለጠባብ ቤቶች ገላግሌ የሆነ ከንፁህ የሲልከን ጥሬ እቃ የተ...,2024-09-26 12:24:51+00:00,photos/@sinayelj_14839.jpg
8,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14835,Baby romper ከላዩ ፈር ውስጡ ኮተን የሆነ\n\n2000 birr\...,2024-09-26 12:24:11+00:00,photos/@sinayelj_14835.jpg
11,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14832,Chicco 5 in 1 mama bag\nኦሪጅናል ማቴሪያል\nበሳይዙ ትልቅ\...,2024-09-26 12:20:22+00:00,photos/@sinayelj_14832.jpg
